## 감성 분석 개요
- 문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법으로 소셜 미디어, 여론조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용되고 있다. 
- 감성 분석은 크게 지도학습 & 비지도학습 방식으로 수행된다. 
- 데이터는 캐글 대회 데이터를 활용하였다. 
- 따라서, 본 포스트에서는 지도학습 기반과 비지도학습 기반의 감성 분석을 실습한다. 


## 데이터 불러오기
- 각각 필요한 데이터를 불러오도록 한다. 

In [2]:
import pandas as pd
review_df = pd.read_csv("data/labeledTrainData.tsv", header = 0, sep="\t", quoting = 3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [3]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


- id: 각 데이터의 ID
- sentiment: 영화평의 결과값 1은 긍정적 평가, 0은 부정적 평가 의미
- review: 영화평의 텍스트임. 

In [4]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

- 특수문자들이 많아서 정규표현식을 활용하여 삭제하도록 한다. 
- 정규표현식에 관한 설명은 다음 글을 참조한다. 
  + [점프 투 파이썬 정규 표현식](https://wikidocs.net/4308)
- `[^a-zA-Z]`의 의미는 영어 대/소문자가 아닌 모든 문자를 찾는 것임. 

In [5]:
import re

# <br> html 태그는 replace 함수를 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />', ' ')

# 파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환 
review_df['review'] = review_df['review'].apply(lambda x : re.sub("[^a-zA-Z]", " ", x))

- 결정 값 클래스인 sentiment 칼럼을 종속변수에 해당하므로 별도 값으로 추출하여 데이터 세트를 만들고, 원본 데이터 세트에서 `id`와 `sentiment` 칼럼을 삭제해 피처 데이터 세트 생성. 

In [6]:
review_df

,id,sentiment,review
0,"""5814_8""",1,With all this stuff going down at the moment ...
1,"""2381_9""",1,The Classic War of the Worlds by Timothy ...
2,"""7759_3""",0,The film starts with a manager Nicholas Bell...
3,"""3630_4""",0,It must be assumed that those who praised thi...
4,"""9495_8""",1,Superbly trashy and wondrously unpretentious ...
...,...,...,...
24995,"""3453_3""",0,It seems like more consideration has gone int...
24996,"""5064_1""",0,I don t believe they made this film Complete...
24997,"""10905_3""",0,Guy is a loser Can t get girls needs to bui...
24998,"""10194_3""",0,This minute documentary Bu uel made in the...


In [7]:
from sklearn.model_selection import train_test_split

class_Y = review_df['sentiment']
feature_X = review_df.drop(['id', 'sentiment'], axis = 1, inplace=False)
print(feature_X)

X_train, X_test, y_train, y_test = train_test_split(feature_X, class_Y, test_size = 0.3, random_state = 156)

X_train.shape, X_test.shape

                                                  review
0       With all this stuff going down at the moment ...
1         The Classic War of the Worlds   by Timothy ...
2       The film starts with a manager  Nicholas Bell...
3       It must be assumed that those who praised thi...
4       Superbly trashy and wondrously unpretentious ...
...                                                  ...
24995   It seems like more consideration has gone int...
24996   I don t believe they made this film  Complete...
24997   Guy is a loser  Can t get girls  needs to bui...
24998   This    minute documentary Bu uel made in the...
24999   I saw this movie as a child and it broke my h...

[25000 rows x 1 columns]


((17500, 1), (7500, 1))

- 학습용 데이터는 17,500개의 리뷰, 테스트용 데이터는 7,500개의 리뷰로 구성됨
- 이제 Pipeline 객체를 활용하여 머신러닝에 적용한다. 
  + 첫번째 방법은 Count 벡터화를 진행한 경우
  + 두번째 방법은 TF-IDF 기반 피처 벡터화를 진행한 경우


### Count 벡터화
- 아래 코드는 `Count` 벡터화를 적용하여 예측 성능을 측정하였다. 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 English, Filtering, Ngram은 (1, 2)로 설정하여 CountVectorization 수행
# LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
                     ('cnt_vect', CountVectorizer(stop_words = 'english', ngram_range=(1, 2))), 
                     ('lr_clf', LogisticRegression(C=10))])

# Pipeline 객체를 이용해 fit(), predict()로 학습/예측 수행, predict_proba()는 roc_auc 때문에 수행. 
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:, 1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8848, ROC-AUC는 0.9509


### TF-IDF 벡터화
- 이번에는 TF-IDF 벡터화를 적용해 다시 예측 성능을 측정한다. 
예제 코드는 위와 거의 같고, 단지 Pipeline에서 CountVectorizer를 TfidfVectorizer로 변경한다. 

In [9]:
# 스톱 워드는 english, filtering, ngram은 (1, 2)로 설정해 TF-IDF 벡터화 수행
# LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
                     ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range = (1, 2))), 
                     ('lr_clf', LogisticRegression(C=10))])
                     
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:, 1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8939, ROC-AUC는 0.9596


## 비지도학습 기반 감성 분석 소개
- 비지도 감성 분석은 `Lexicon`을 기반으로 한다. 위 예제는 사전에 결정된 레이블 값을 가지고 있지만, 현실에서는 그렇지 않다. 
- 그럴 경우 `Lexicon`은 유용하게 사용될 수 있다. 


## 감성사전 개요
- `Lexicon`은 일반적으로 어휘집을 의미하나, 대개 감성 어휘 사전을 말한다. 감성 사전은 긍정(`Positive`) 또는 부정(`Negative`)감성의 정도를 의미하는 수치를 가지며, 이를 `Polarity Score`라 부른다. 
- `NLP`에서 제공하는 `WordNet` 모듈은 시맨틱 분석을 제공하는 어휘 사전인데, 시맨틱은 간단히 표현하면 `문맥상의 의미`를 말한다. 
- `WordNet`은 다양한 상황에서 같은 어휘라도 다르게 사용되는 어휘의 시맨틱 정보를 제공하며, 이를 위해 각각의 품사(명사, 동사, 형용사, 부사 등)으로 구성된 개별 단어를 Synset(`Sets of cognitive synonums`)이라는 개념을 이용해 표현한다. 
- 감성사전의 종류는 크게 3가지가 존재한다. 
  + `SentiWordNet`은 `Synset`별로 긍정 감성 지수, 부정 감성 지수, 객관성 지수를 수치로 표현함. 
  + Vader: 주로 소셜 미디어의 텍스트에 대한 감성 분석을 제공하기 위한 패키지이며, 뛰어난 감성 분석 결과를 제공하며, 대용량 텍스트 데이터에서 잘 사용되는 패키지임. 



## SentiWordNet 감성 분석
- `NLTK` 패키지를 활용하여 `WordNet`을 활용합니다. 
- 우선 시간 측정을 위해 아래와 같은 함수를 정의합니다. 

In [10]:
import time
import datetime
def bench_mark(start):
  sec = time.time() - start
  times = str(datetime.timedelta(seconds=sec)).split(".")
  times = times[0]
  print(times)

In [11]:
start = time.time()

import nltk
nltk.download('all')

bench_mark(start)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       tagge

0:01:14


[nltk_data]    |   Unzipping corpora\ycoe.zip.
[nltk_data]    | 
[nltk_data]  Done downloading collection all


- 이 때, `WordNet` 모듈을 불러와서 `present` 단어에 대한 `Synset`을 추출합니다. 

In [12]:
from nltk.corpus import wordnet as wn 

the_word = "present"

# 'present'라는 단어로 `wordnet`의 `synsets` 생성
synsets = wn.synsets(the_word)
print('synsets() 반환 type :', type(synsets))
print('synsets() 반환 값 개수 :', len(synsets))
print('synsets() 반환 값 :', synsets)

synsets() 반환 type : <class 'list'>
synsets() 반환 값 개수 : 18
synsets() 반환 값 : [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


총 18개의 서로 다른 semantic을 가지는 `synset 객체가 반환되었습니다. 
이를 이제 `synset` 객체가 가지는 여러 가지 속성을 살펴보도록 합니다. 

In [13]:
for synset in synsets:
  print("#### Synset name :", synset.name(), "####")
  print("POS :", synset.lexname())
  print('Definition:', synset.definition())
  print('Lemmas:', synset.lemma_names())

#### Synset name : present.n.01 ####
POS : noun.time
Definition: the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas: ['present', 'nowadays']
#### Synset name : present.n.02 ####
POS : noun.possession
Definition: something presented as a gift
Lemmas: ['present']
#### Synset name : present.n.03 ####
POS : noun.communication
Definition: a verb tense that expresses actions or states at the time of speaking
Lemmas: ['present', 'present_tense']
#### Synset name : show.v.01 ####
POS : verb.perception
Definition: give an exhibition of to an interested audience
Lemmas: ['show', 'demo', 'exhibit', 'present', 'demonstrate']
#### Synset name : present.v.02 ####
POS : verb.communication
Definition: bring forward and present to the mind
Lemmas: ['present', 'represent', 'lay_out']
#### Synset name : stage.v.01 ####
POS : verb.creation
Definition: perform (a play), especially on a stage
Lemmas: ['stage', 'present', 'represent']
#### Synset na

- 각각에 호출된 단어를 확인하도록 합니다. 


### WordNet 유사도 측정
- `WordNet`은 어떤 어휘와 다른 어휘 간의 관계를 유사도로 나타낼 수 있다. 
- `synset`객체는 단어 간의 유사도를 나타내기 위해 `path_similarity()` 제공한다. 


In [14]:
plant = wn.synset('plant.n.01')
rabbit = wn.synset('rabbit.n.01')
lion = wn.synset('lion.n.01')
horse = wn.synset('horse.n.01')
dog = wn.synset('dog.n.01')

entities = [plant, rabbit, lion, horse, dog]
similarities = []
entity_names = [entity.name().split('.')[0] for entity in entities]

# 단어별 synset을 반복하면서 다른 단어의 `synset`과 유사도를 측정함
for entity in entities:
  similarity = [round(entity.path_similarity(compared_entity), 2) for compared_entity in entities]
  similarities.append(similarity)

# 개별 단어별 synset과 다른 단어의 `synset`과의 유사도를 `DataFrame` 형태로 저장
similarity_df = pd.DataFrame(similarities, columns = entity_names, index = entity_names)
similarity_df

,plant,rabbit,lion,horse,dog
plant,1.00,0.07,0.06,0.06,0.10
rabbit,0.07,1.00,0.12,0.12,0.14
lion,0.06,0.12,1.00,0.11,0.17
horse,0.06,0.12,0.11,1.00,0.12
dog,0.10,0.14,0.17,0.12,1.00


### SentiSynset 감성지수
- `SentiSynset` 객체는 단어의 감성을 나타내는 감성 지수와 객관성을(감성과 반대) 나타내는 객관성 지수를 가지고 있다. 
  + 어떤 단어가 감성적이지 않으면 객관성 지수는 1 / 감성 지수는 0 이렇게 구분된다. 


In [15]:
import nltk

from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset('father.n.01')
print('father 긍정감성 지수: ', father.pos_score())
print('father 부정감성 지수: ', father.neg_score())
print('father 객관성 지수: ', father.obj_score())
print('\n')
fabulous = swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수: ', fabulous.pos_score())
print('fabulous 부정감성 지수: ', fabulous.neg_score())
print('fabulous 객관성 지수: ', fabulous.obj_score())

father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0


fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125
fabulous 객관성 지수:  0.0


- 위 결과값에 대해 비교해보도록 한다. 

### SentiWordNet 영화 감상평 감성 분석
- 문서`(Document)`를 문장`(Sentence)` 단위로 분해
- 다시 문장을 단어`(Word)` 단위로 토큰화하고 품사 태깅
- 품사 태깅된 단어 기반으로 `synset` 객체와 `senti_synset` 객체를 생성
- `Senti_synset`에서 긍정 감성/부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않을 때는 부정 감성으로 결정
- `WordNet`을 이용해 문서를 다시 단어로 토큰화한 뒤 어근 추출(`Lemmatization`)과 품사 태깅(`POS Tagging`)을 적용해야 합니다. 
- 먼저 품사 태깅을 수행하는 내부 함수 생성합니다. 

In [16]:
from nltk.corpus import wordnet as wn

# 간단한 NLTK PennTreebank Tag를 기반으로 WordNet 기반의 품사 Tag로 변환
def penn_to_wn(tag):
  if tag.startswith('J'):
    return wn.ADJ
  elif tag.startswith('N'):
    return wn.NOUN
  elif tag.startswith('R'):
    return wn.ADV
  elif tag.startswith('V'):
    return wn.VERB

- 이제 문서를 문장, 단어 토큰, 품사 태킹 후에 `SentiSynset` 클래스를 생성하고 `Polarity Score`를 합산하는 함수를 생성한다. 
- 각 단어의 긍정 감성 지수와 부정 감성 지수를 모두 합한 총 감성 지수가 0 이상일 경우 긍정 감성, 그렇지 않을 경우 부정 감성으로 예측한다. 

In [17]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화 
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출  
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word , tag in tagged_sentence:
            
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN , wn.ADJ, wn.ADV):
                continue                   
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성. 
            synsets = wn.synsets(lemma , pos=wn_tag)
            if not synsets:
                continue
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산. 
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())           
            tokens_count += 1
    
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0 :
        return 1
    
    return 0

- 이렇게 생성한 `swn_polarity(text)` 함수를 IMDB 감상평의 개별 문서에 적용하여 긍정 및 부정 감성을 예측한다. 


In [18]:
start = time.time()

review_df['preds'] = review_df['review'].apply(lambda x: swn_polarity(x))
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

bench_mark(start)

0:01:50


`SentiWordNet`의 감성 분석 예측 성능을 살펴보도록 한다. 

In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix(y_target, preds))
print("정확도:", np.round(accuracy_score(y_target, preds), 4))
print("정밀도:", np.round(precision_score(y_target, preds), 4))
print("재현율:", np.round(recall_score(y_target, preds), 4))

[[7669 4831]
 [3635 8865]]
정확도: 0.6614
정밀도: 0.6473
재현율: 0.7092
